In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cma
from scipy.optimize import differential_evolution
from deap import base, creator, tools, algorithms
import random
from scipy.optimize import dual_annealing
import pyswarms as ps
from openpyxl import load_workbook


## Function to apply convolution principle to calcuate runoff 

In [70]:
def hydrograph_convolution(unit_hydrograph, excess_rainfall):
    """
    Calculates the Direct Runoff Hydrograph (DRH) using convolution.

    Args:
        unit_hydrograph (list ): Unit Hydrograph Ordinates values.
        excess_rainfall (list ): Excess Rainfall Hyetograph values.

    Returns:
        numpy array: Direct Runoff Hydrograph values.
    """

    direct_runoff = np.convolve(unit_hydrograph, excess_rainfall, mode='full')
    return direct_runoff

## Function to calculate ordinates of unit hydrographs

In [73]:
def unit_hydrograph_ordinates(R, T, K, delta_t):
    '''
    To calculate ordinates of given traingular RTK hydrograph
     Args:
         R= ratio of rdii volume to rainfall volume
         T= time from onset of rainfall to the peak of unit hydrograph (seconds)
         K= ratio of time to recession of the unit hydrograph
         delta_t = rainfall data time step in seconds
    return:
        list of ordinates value of unit hydrographs
    '''
    # Calculate the number of ordinates
    num_ordinates = int((T + K * T) / delta_t)
    
    # Initialize list to store the ordinates
    ordinates = []

    # Loop through each ordinate
    for j in range(1, num_ordinates + 1):
        # Calculate tau
        tau = (j - 0.5) * delta_t
        
        # Determine the value of f based on the conditions
        if tau <= T:
            f = tau / T
        elif T < tau <= T + K * T:
            f = 1 - (tau - T) / (K * T)
        else:
            f = 0

        # Calculate the unit hydrograph ordinate
        UH = 2 * R * f / (T + K * T)
        ordinates.append(UH)

    return ordinates

## Function to plot all hydrograph

In [76]:
def plot_unit_hydrographs_with_sum(params, delta_t, filename=None):
    """
    Plots multiple unit hydrographs based on given R, T, K parameter sets and their summation.
    
    Args:
        params (list of tuples): Each tuple contains (R, T, K) values.
        delta_t (float): Time step in seconds.
    """
    plt.figure(figsize=(10, 6))
    
    # Store all ordinates and time values
    all_ordinates = []
    max_length = 0  # To find the maximum length of the ordinates

    # Iterate over each set of R, T, K values
    for i, (R, T, K) in enumerate(params, start=1):
        # Calculate the ordinates for the current R, T, K
        ordinates = unit_hydrograph_ordinates(R, T, K, delta_t)
        
        # Add 0 at the beginning and end of the ordinates list
        ordinates = [0] + ordinates + [0]
        
        # Keep track of the maximum length of ordinates
        max_length = max(max_length, len(ordinates))
        
        # Append ordinates to all_ordinates for summation
        all_ordinates.append(ordinates)
        
        # Generate the time values for the x-axis, starting from 0
        time_values = [j * delta_t / 3600 for j in range(len(ordinates))]
        
        # Plot the unit hydrograph for the current R, T, K
        plt.plot(time_values, ordinates, linestyle='-', label=f'UH{i} (R={R}, T={T}, K={K})')

    # Pad the shorter ordinates with zeros to match the maximum length
    padded_ordinates = []
    for ordinates in all_ordinates:
        padded_ordinates.append(ordinates + [0] * (max_length - len(ordinates)))

    # Calculate the summation of all ordinates element-wise
    summed_ordinates = np.sum(padded_ordinates, axis=0)
    
    # Generate the time values for the x-axis for the summed plot
    time_values = [i * delta_t / 3600 for i in range(max_length)]
    
    # Plot the summed unit hydrograph
    plt.plot(time_values, summed_ordinates, linestyle='--', color='black', label='Sum of All UH Sets')
    
    # Plot settings
    plt.xlabel('Time [Hrs]')
    plt.ylabel('[1/sec]')
    plt.title('Unit Hydrographs')
    plt.legend()
    plt.grid(True)
    
    # Set the x and y axis limits to start at 0
    plt.xlim(left=0)
    plt.ylim(bottom=0)
       

    
    # Save the plot if filename is provided
    if filename:
        plt.savefig(filename)
        print(f"Plot saved as {filename}")

    plt.show()

### Function to add flow due each unit hydrograph to get total flow

In [79]:
def add_flow(*arrays):
    # Find the length of the longest array
    max_length = max(len(arr) for arr in arrays)
    
    # Pad each array with zeros to match the length of the longest array
    padded_arrays = [np.pad(arr, (0, max_length - len(arr)), 'constant') for arr in arrays]
    
    # Sum the padded arrays element-wise
    result = np.sum(padded_arrays, axis=0)
    
    return result.tolist()

# Function to calculate toal RDII and Plot Rainfall and RDII

## Calculating and plotting RDII

In [83]:
# def RDII_calculation_and_plot(params, delta_t, rainfall, Area, filename = None):
#     # Unpack the tuple of parameters
#     (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params
    
#     # Calculate unit hydrograph ordinates for each set of parameters
#     uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
#     uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
#     uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)
    
#     # Perform convolution with rainfall data
#     Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
#     Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
#     Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

#     # Convert flow from inch/sec to cubic feet per second (cfs)
#     Q1_cfs = Q1_inch_sec * Area* 43560 / 12
#     Q2_cfs = Q2_inch_sec * Area* 43560 / 12
#     Q3_cfs = Q3_inch_sec * Area* 43560 / 12

#     # Total flow by adding the flows from each hydrograph
#     total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)
    
#     # Generate the time axis in minutes
#     time_values = [i * delta_t / 3600 for i in range(len(total_flow))]

#     # Plotting
#     fig, ax1 = plt.subplots(figsize=(10, 6))

#     # Plot total RDII as a line plot on the first y-axis
#     ax1.plot(time_values, total_flow, label="RDII (cfs)", color='b')
#     ax1.set_xlabel('Time (Hrs)')
#     ax1.set_ylabel('RDII (cfs)', color='b')
#     ax1.tick_params(axis='y', labelcolor='b')
#     ax1.legend(loc='upper left')
#     ax1.grid(True)

#     # Create a secondary y-axis for rainfall and align scales
#     ax2 = ax1.twinx()
#     rainfall_time_values = [i * delta_t / 3600 for i in range(len(rainfall))]
#     ax2.bar(rainfall_time_values, rainfall, width=delta_t / 3600 * 0.8, color='gray', alpha=0.5, label="Rainfall (in)")
#     ax2.set_ylabel('Rainfall (in)', color='gray')
#     ax2.tick_params(axis='y', labelcolor='gray')
#     ax2.legend(loc='upper right')

#     # Set both y-axes to the same scale
#     y_min = 0
#     y_max = max(max(total_flow), max(rainfall)) * 1.2  # add 10% padding to the max value
#     ax1.set_ylim(y_min, y_max)
#     ax2.set_ylim(y_min, y_max)
#     # Set the x and y axis limits to start at 0
#     plt.xlim(left=0)
#     plt.ylim(bottom=0)

#     plt.title('Total RDII and Rainfall Over Time')
    

#      # Save the plot if filename is provided
#     if filename:
#         plt.savefig(filename)
#         print(f"Plot saved as {filename}")

#     plt.show()
    
#     return total_flow

### Plotting sim RDII (also calculating) and Obs RDII 

In [15]:
def RDII_calculation_and_plot(params, delta_t, rainfall, Area, filename=None, obs_rdii=None):
    # Unpack the tuple of parameters
    (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params
    
    # Calculate unit hydrograph ordinates for each set of parameters
    uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
    uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
    uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)
    
    # Perform convolution with rainfall data
    Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
    Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
    Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

    # Convert flow from inch/sec to cubic feet per second (cfs)
    Q1_cfs = Q1_inch_sec * Area * 43560 / 12
    Q2_cfs = Q2_inch_sec * Area * 43560 / 12
    Q3_cfs = Q3_inch_sec * Area * 43560 / 12

    # Total flow by adding the flows from each hydrograph
    total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)

    # Generate the time axis in hours
    time_values = [i * delta_t / 3600 for i in range(len(total_flow))]

    # Handle the case where obs_rdii is provided
    if obs_rdii is not None:
        # Pad the shorter array with zeros to match the length of the longer one
        max_length = max(len(total_flow), len(obs_rdii))
        padded_sim_rdii = np.pad(total_flow, (0, max_length - len(total_flow)), 'constant')
        padded_obs_rdii = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), 'constant')
        padded_time = [i * delta_t / 3600 for i in range(max_length)]  # Extend time to match max_length
    else:
        padded_sim_rdii = total_flow
        padded_obs_rdii = None
        padded_time = time_values

    # Plotting
    fig, ax1 = plt.subplots(figsize=(10, 6))

    # Plot total RDII (simulated) on the first y-axis
    ax1.plot(padded_time, padded_sim_rdii, label="Simulated RDII", color='b', linestyle='--', linewidth=1.5)
    ax1.set_xlabel('Time (Hrs)')
    ax1.set_ylabel('RDII (cfs)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    ax1.legend(loc='upper left')
    ax1.grid(True)

    # Plot observed RDII if available
    if obs_rdii is not None:
        ax1.plot(padded_time, padded_obs_rdii, label="Observed RDII ", color='r', linestyle='-', linewidth=1.5)
        ax1.legend(loc='upper left')

    # Create a secondary y-axis for rainfall
     # Plot rainfall as a bar chart
    #ax2 = ax1.twinx()
   #   # Set the y-axis limits using the maximum total flow
   #  y_min = 0
   #  y_max = max(
   #      #max_total_flow, 
   #      max(rainfall),  # Convert rainfall to flow equivalent
   #      max(obs_rdii) if obs_rdii is not None else 0
   #  ) * 1.2  # Add 20% padding

   #  # Plot the rainfall data as an inverted pulse (stem plot)
   #  markerline, stemlines, baseline = ax2.stem(time_values, rainfall, linefmt='blue', markerfmt=' ', basefmt=' ', label='Rainfall')
   #  plt.setp(stemlines, 'color', 'blue', 'alpha', 0.5)
   #  plt.setp(markerline, 'color', 'blue', 'alpha', 0.5)

   #  # Invert the y-axis for rainfall data
   #  ax2.set_ylim(0.6, 0)
   # # ax2.set_ylim(y_max, 0)  # Inverted y-axis (higher values at the bottom)

   #  # Set the y-axis limits using the maximum total flow
   #  ax1.set_ylim(0, max_total_flow * 1.2)

    ax2 = ax1.twinx()
    rainfall_time_values = [i * delta_t / 3600 for i in range(len(rainfall))]
    # Plot the rainfall data as an inverted pulse (stem plot)
    markerline, stemlines, baseline = ax2.stem(rainfall_time_values , rainfall, linefmt='blue', markerfmt=' ', basefmt=' ', label='Rainfall')
    plt.setp(stemlines, 'color', 'blue', 'alpha', 0.5)
    plt.setp(markerline, 'color', 'blue', 'alpha', 0.5)
    
    #ax2.bar(rainfall_time_values, rainfall, width=delta_t / 3600 * 0.8, color='gray', alpha=0.5, label="Rainfall")
    ax2.set_ylabel('Rainfall (in)', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')
    ax2.legend(loc='upper right')

    # Set the y-axis limits to the same range
    y_min = 0
    y_max = max(
        max(total_flow), 
        max(rainfall), 
        max(obs_rdii) if obs_rdii is not None else 0
    ) * 1.2  # Add 20% padding
    ax1.set_ylim(y_min, y_max)
    ax2.set_ylim(0.6,0)

    # Ensure the x-axis starts at 0
    plt.xlim(left=0)

    plt.title('Total RDII and Rainfall Over Time')

    # Save the plot if filename is provided
    if filename:
        plt.savefig(filename)
        print(f"Plot saved as {filename}")

    plt.show()

    return total_flow

### Plotting for Multiple Run

In [8]:
def RDII_combined_plot(all_params, delta_t, rainfall, Area, obs_rdii=None):
    """
    Combines RDII plots for multiple parameter sets into one plot.

    Parameters:
    - all_params: List of parameter sets (from multiple runs).
    - delta_t: Time step in seconds.
    - rainfall: Rainfall time series (in inches).
    - Area: Catchment area in acres.
    - obs_rdii: Observed RDII time series in cfs (optional).

    Returns:
    - predicted_flows: List of predicted RDII flows in m³/s for each run
    """
    fig, ax1 = plt.subplots(figsize=(12, 8))

    time_values = [i * delta_t / 3600 for i in range(len(rainfall))]

    predicted_flows = []
    max_total_flow = 0

    for idx, params in enumerate(all_params):
        (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params

        uh1 = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
        uh2 = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
        uh3 = unit_hydrograph_ordinates(R3, T3, K3, delta_t)

        Q1 = hydrograph_convolution(uh1, rainfall)
        Q2 = hydrograph_convolution(uh2, rainfall)
        Q3 = hydrograph_convolution(uh3, rainfall)

        # Convert to cfs first, then to m³/s
        total_flow_cfs = add_flow(
            Q1 * Area * 43560 / 12,
            Q2 * Area * 43560 / 12,
            Q3 * Area * 43560 / 12
        )
        total_flow_m3s = np.array(total_flow_cfs) * 0.0283168

        predicted_flows.append(total_flow_m3s)
        max_total_flow = max(max_total_flow, max(total_flow_m3s))

        max_length = max(len(total_flow_m3s), len(obs_rdii) if obs_rdii is not None else 0)
        padded_flow = np.pad(total_flow_m3s, (0, max_length - len(total_flow_m3s)), 'constant')

        ax1.plot(
            [i * delta_t / 3600 for i in range(max_length)],
            padded_flow,
            label=f"Sim. RDII Run {idx+1}",
            linestyle='--'
        )

    # Plot observed RDII (convert from cfs to m³/s)
    if obs_rdii is not None:
        padded_obs = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), 'constant')
        obs_m3s = np.array(padded_obs) * 0.0283168
        ax1.plot(
            [i * delta_t / 3600 for i in range(max_length)],
            obs_m3s,
            label="Observed RDII",
            color='r',
            linewidth=2
        )

    # Convert rainfall to mm for plotting
    rainfall_mm = np.array(rainfall) * 25.4
    ax2 = ax1.twinx()
    markerline, stemlines, baseline = ax2.stem(time_values, rainfall_mm, linefmt='blue', markerfmt=' ', basefmt=' ', label='Rainfall')
    plt.setp(stemlines, 'color', 'blue', 'alpha', 0.5)
    plt.setp(markerline, 'color', 'blue', 'alpha', 0.5)
    ax2.set_ylim(max(rainfall_mm) * 1.2, 0)

    ax1.set_ylim(0, max_total_flow * 1.2)
    plt.xlim(left=0)

    ax1.set_xlabel('Time (Hours)')
    ax1.set_ylabel('RDII (m³/s)')
    ax2.set_ylabel('Rainfall (mm)', color='blue')
    ax2.tick_params(axis='y', labelcolor='blue')
    ax1.legend(loc='upper right')
    ax2.legend(loc='upper left')
    ax1.grid(True)
    ax1.set_title('Combined RDII and Rainfall Plot for Multiple Runs')

    plt.show()
    
    return predicted_flows


In [8]:
# def RDII_combined_plot(all_params, delta_t, rainfall, Area, obs_rdii=None):
#     """
#     Combines RDII plots for multiple parameter sets into one plot.

#     Parameters:
#     - all_params: List of parameter sets (from multiple runs).
#     - delta_t: Time step in seconds.
#     - rainfall: Rainfall time series (in inches).
#     - Area: Catchment area in acres.
#     - obs_rdii: Observed RDII time series (optional).

#     Returns:
#     - None
#     """
#     fig, ax1 = plt.subplots(figsize=(12, 8))

#     # Generate a time axis for the rainfall and RDII
#     time_values = [i * delta_t / 3600 for i in range(len(rainfall))]

#     # List to store predicted flows for each run
#     predicted_flows = []

#     # Variable to track the maximum total flow
#     max_total_flow = 0

#     # Loop through each parameter set and calculate RDII
#     for idx, params in enumerate(all_params):
#         (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params

#         # Calculate unit hydrograph ordinates for each parameter set
#         uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
#         uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
#         uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)

#         # Perform convolution with rainfall data
#         Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
#         Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
#         Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

#         # Convert flow from inch/sec to cubic feet per second (cfs)
#         Q1_cfs = Q1_inch_sec * Area * 43560 / 12
#         Q2_cfs = Q2_inch_sec * Area * 43560 / 12
#         Q3_cfs = Q3_inch_sec * Area * 43560 / 12

#         # Total flow by adding the flows from each hydrograph
#         total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)

#         # Store the total flow in the predicted_flows list
#         predicted_flows.append(total_flow)

#         # Update the maximum total flow
#         max_total_flow = max(max_total_flow, max(total_flow))

#         # Pad the total_flow if necessary
#         max_length = max(len(total_flow), len(obs_rdii) if obs_rdii is not None else 0)
#         padded_total_flow = np.pad(total_flow, (0, max_length - len(total_flow)), 'constant')

#         # Add the simulated RDII to the plot
#         ax1.plot(
#             [i * delta_t / 3600 for i in range(max_length)], 
#             padded_total_flow, 
#             label=f"Sim. RDII Run {idx+1}", 
#             linestyle='--'
#         )

#     # Plot observed RDII if available
#     if obs_rdii is not None:
#         max_length = max(len(obs_rdii), len(total_flow))
#         padded_obs_rdii = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), 'constant')
#         ax1.plot(
#             [i * delta_t / 3600 for i in range(max_length)], 
#             padded_obs_rdii, 
#             label="Observed RDII", 
#             color='r', 
#             linewidth=2
#         )

#     # Plot rainfall as a bar chart
#     ax2 = ax1.twinx()
#     # ax2.bar(
#     #     time_values, 
#     #     rainfall, 
#     #     width=delta_t / 3600 * 0.8, 
#     #     color='gray', 
#     #     alpha=0.5, 
#     #     label="Rainfall"
#     # )

#     # Set the y-axis limits using the maximum total flow
#     y_min = 0
#     y_max = max(
#         max_total_flow, 
#         max(rainfall),  # Convert rainfall to flow equivalent
#         max(obs_rdii) if obs_rdii is not None else 0
#     ) * 1.2  # Add 20% padding

#     # Plot the rainfall data as an inverted pulse (stem plot)
#     markerline, stemlines, baseline = ax2.stem(time_values, rainfall, linefmt='blue', markerfmt=' ', basefmt=' ', label='Rainfall')
#     plt.setp(stemlines, 'color', 'blue', 'alpha', 0.5)
#     plt.setp(markerline, 'color', 'blue', 'alpha', 0.5)

#     # Invert the y-axis for rainfall data
#     ax2.set_ylim(0.6, 0)
#    # ax2.set_ylim(y_max, 0)  # Inverted y-axis (higher values at the bottom)

#     # Set the y-axis limits using the maximum total flow
#     ax1.set_ylim(0, max_total_flow * 1.2)


#     # ax1.set_ylim(y_min, y_max)
#     # ax2.set_ylim(y_min, y_max)
#     # ax2.set_ylim(y_min, y_max)

#     # Ensure the x-axis starts at 0
#     plt.xlim(left=0)

#     # Axis labels and legends
#     ax1.set_xlabel('Time (Hours)')
#     ax1.set_ylabel('RDII (cfs)')
#     ax2.set_ylabel('Rainfall (in)', color='blue')
#     ax2.tick_params(axis='y', labelcolor='blue')
#     ax1.legend(loc='upper right')
#     ax2.legend(loc='upper left')
#     ax1.grid(True)
#     ax1.set_title('Combined RDII and Rainfall Plot for Multiple Runs')

#     plt.show()
    
#     return predicted_flows




In [91]:
# def RDII_calculation_and_plot(params, delta_t, rainfall, Area, filename=None, obs_rdii=None):
#     # Unpack the tuple of parameters
#     (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params
    
#     # Calculate unit hydrograph ordinates for each set of parameters
#     uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
#     uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
#     uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)
    
#     # Perform convolution with rainfall data
#     Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
#     Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
#     Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

#     # Convert flow from inch/sec to cubic feet per second (cfs)
#     Q1_cfs = Q1_inch_sec * Area * 43560 / 12
#     Q2_cfs = Q2_inch_sec * Area * 43560 / 12
#     Q3_cfs = Q3_inch_sec * Area * 43560 / 12

#     # Total flow by adding the flows from each hydrograph
#     total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)

#     # Convert inputs to numpy arrays
#     total_flow = np.array(total_flow)
#     if obs_rdii is not None:
#         obs_rdii = np.array(obs_rdii)

#         # Truncate or pad sim_rdii (total_flow) to match the length of obs_rdii
#         if len(total_flow) > len(obs_rdii):
#             total_flow = total_flow[:len(obs_rdii)]
#         elif len(total_flow) < len(obs_rdii):
#             total_flow = np.pad(total_flow, (0, len(obs_rdii) - len(total_flow)), mode='constant')

#         # Generate the time axis based on obs_rdii length
#         time_values = [i * delta_t / 3600 for i in range(len(obs_rdii))]
#     else:
#         # Use total_flow's length for time_values if obs_rdii is not provided
#         time_values = [i * delta_t / 3600 for i in range(len(total_flow))]

#     # Plotting
#     fig, ax1 = plt.subplots(figsize=(10, 6))

#     # Plot total RDII (simulated) on the first y-axis
#     ax1.plot(time_values, total_flow, label="Simulated RDII", color='b', linestyle='--', linewidth=1.5)
#     ax1.set_xlabel('Time (Hrs)')
#     ax1.set_ylabel('RDII (cfs)', color='b')
#     ax1.tick_params(axis='y', labelcolor='b')
#     ax1.legend(loc='upper left')
#     ax1.grid(True)

#     # Plot observed RDII if available
#     if obs_rdii is not None:
#         ax1.plot(time_values, obs_rdii, label="Observed RDII", color='r', linestyle='-', linewidth=1.5)
#         ax1.legend(loc='upper left')

#     # Create a secondary y-axis for rainfall
#     ax2 = ax1.twinx()
#     rainfall_time_values = [i * delta_t / 3600 for i in range(len(rainfall))]
#     ax2.bar(rainfall_time_values, rainfall, width=delta_t / 3600 * 0.8, color='gray', alpha=0.5, label="Rainfall")
#     ax2.set_ylabel('Rainfall (in)', color='gray')
#     ax2.tick_params(axis='y', labelcolor='gray')
#     ax2.legend(loc='upper right')

#     # Set the y-axis limits to the same range
#     y_min = 0
#     y_max = max(
#         max(total_flow), 
#         max(rainfall), 
#         max(obs_rdii) if obs_rdii is not None else 0
#     ) * 1.2  # Add 20% padding
#     ax1.set_ylim(y_min, y_max)
#     ax2.set_ylim(y_min, y_max)

#     # Ensure the x-axis starts at 0
#     plt.xlim(left=0)

#     plt.title('Total RDII and Rainfall Over Time')

#     # Save the plot if filename is provided
#     if filename:
#         plt.savefig(filename)
#         print(f"Plot saved as {filename}")

#     plt.show()

#     return total_flow


## Calculating RDII

In [94]:
def RDII_calculation(params, delta_t, rainfall, Area, filename = None):
   # Unpack the tuple of parameters
    (R1, T1, K1), (R2, T2, K2), (R3, T3, K3) = params
       #Convert flat array back to parameter tuples
    #R1, T1, K1, R2, T2, K2, R3, T3, K3 = params_flat
    
    # R1, T1_scaled, K1, R2, T2_scaled, K2, R3, T3_scaled, K3 = params_flat
    # T1, T2, T3 = T1_scaled * 10000, T2_scaled * 10000, T3_scaled * 10000
    
    # params = [(R1, T1, K1), (R2, T2, K2), (R3, T3, K3)]
    
    # Calculate unit hydrograph ordinates for each set of parameters
    uh1_ordinates = unit_hydrograph_ordinates(R1, T1, K1, delta_t)
    uh2_ordinates = unit_hydrograph_ordinates(R2, T2, K2, delta_t)
    uh3_ordinates = unit_hydrograph_ordinates(R3, T3, K3, delta_t)
    
    # Perform convolution with rainfall data
    Q1_inch_sec = hydrograph_convolution(uh1_ordinates, rainfall)
    Q2_inch_sec = hydrograph_convolution(uh2_ordinates, rainfall)
    Q3_inch_sec = hydrograph_convolution(uh3_ordinates, rainfall)

    # Convert flow from inch/sec to cubic feet per second (cfs)
    Q1_cfs = Q1_inch_sec * Area* 43560 / 12
    Q2_cfs = Q2_inch_sec * Area* 43560 / 12
    Q3_cfs = Q3_inch_sec * Area* 43560 / 12

    # Total flow by adding the flows from each hydrograph
    total_flow = add_flow(Q1_cfs, Q2_cfs, Q3_cfs)
    
    
    return total_flow

## Calcuation of R

In [97]:
def R_calc(rainfall_series, rdii_series, delta_t, Area_acres):
    """
    Calculate R, the fraction of RDII volume to total rainfall volume.

    Args:
        rainfall_series (array-like): Time series of rainfall data(inch).
        rdii_series (array-like): Time series of RDII data (cfs).
        delta_t (float): Time step in seconds.
        Area_acres(float) : Area in acres 

    Returns:
        float: The calculated R value.
    """
    # Convert rainfall depth (inches) to volume in cubic feet
    total_rainfall_depth_in = np.sum(rainfall_series)  # Sum of rainfall in inches
    total_rainfall_volume_cf = total_rainfall_depth_in * Area_acres * 3630
       
    # Calculate total rainfall volume and RDII volume
    total_rdii_volume = np.trapz(rdii_series, dx= delta_t) # in 
   
   
    # Calculate R as the ratio of RDII volume to total rainfall volume
    R = total_rdii_volume / total_rainfall_volume_cf if total_rainfall_volume_cf != 0 else 0

    return R

# Calculation of Fitnnes function

In [100]:
# def fitness(obs_rdii, sim_rdii, delta_t, weight_rmse, weight_r2, weight_pbias, weight_nse):
#     """
#     Calculate a combined fitness score using RMSE and Percent Volume Difference (Vol. Diff).

#     Args:
#         obs_rdii (array-like): Observed RDII values.
#         sim_rdii (array-like): Simulated RDII values.
#         delta_t (float): Time interval in seconds.
#         weight_rmse (float): Weight for RMSE in the combined score.
#         weight_vol_diff (float): Weight for Volume Difference in the combined score.

#     Returns:
#         float: Combined fitness score (lower is better).
#     """
#     # Convert inputs to numpy arrays
#     obs_rdii = np.array(obs_rdii)
#     sim_rdii = np.array(sim_rdii)
    
#     # Truncate or pad sim_rdii to match the length of obs_rdii
#     if len(sim_rdii) > len(obs_rdii):
#         sim_rdii = sim_rdii[:len(obs_rdii)]
#     elif len(sim_rdii) < len(obs_rdii):
#         sim_rdii = np.pad(sim_rdii, (0, len(obs_rdii) - len(sim_rdii)), mode='constant')
    
#     # Check for invalid outputs
#     if sim_rdii.size == 0 or np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
#         print("Invalid sim_rdii generated, returning large penalty.")
#         return float('inf')

#     # Replace NaNs or infinite values with large penalties
#     obs_rdii = np.nan_to_num(obs_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
#     sim_rdii = np.nan_to_num(sim_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
    
#     # Calculate Root Mean Square Error (RMSE)
#     rmse = np.sqrt(np.mean((obs_rdii - sim_rdii) ** 2))
    
#     # Calculate R² (Coefficient of Determination)
#     ss_total = np.sum((obs_rdii - np.mean(obs_rdii)) ** 2)
#     ss_residual = np.sum((obs_rdii - sim_rdii) ** 2)
#     r2 = 1 - (ss_residual / ss_total) if ss_total != 0 else 0

#     # Calculate Percent Bias (PBIAS)
#     pbias = 100 * np.sum(sim_rdii - obs_rdii) / np.sum(obs_rdii) if np.sum(obs_rdii) != 0 else float('inf')

#     # Calculate Nash-Sutcliffe Efficiency (NSE)
#     nse = 1 - (ss_residual / ss_total) if ss_total != 0 else -float('inf')

#     # Invert R² and NSE to minimize (maximize by minimizing their negative deviation from 1)
#     r2_penalty = abs(1 - r2)
#     nse_penalty = abs(1 - nse)

#     # Normalize Percent Bias to a positive value
#     pbias_penalty = abs(pbias)
    
    
#     # # Calculate the volumes for observed and simulated RDII (area under the curve)
#     # obs_volume = np.trapz(obs_rdii, dx=delta_t)
#     # sim_volume = np.trapz(sim_rdii, dx=delta_t)
    
#     # # Calculate Percent Volume Difference
#     # if obs_volume != 0:
#     #     vol_diff = 100 * (sim_volume - obs_volume) / obs_volume
#     # else:
#     #     vol_diff = float('inf')  # Large penalty if observed volume is zero

#     # # Normalize the vol_diff to a positive value (minimize absolute deviation)
#     # vol_diff_abs = abs(vol_diff)

#     #  # Combined fitness score (weighted sum of all criteria)
#     combined_score = (weight_rmse * rmse +weight_r2 * r2_penalty + weight_pbias * pbias_penalty + weight_nse * nse_penalty)

#     return combined_score


In [102]:
def fitness(obs_rdii, sim_rdii, delta_t, weight_rmse, weight_r2, weight_pbias, weight_nse):
    """
    Calculate a combined fitness score using normalized RMSE, R², Percent Bias (PBIAS), and NSE.

    Args:
        obs_rdii (array-like): Observed RDII values.
        sim_rdii (array-like): Simulated RDII values.
        delta_t (float): Time interval in seconds.
        weight_rmse (float): Weight for RMSE in the combined score.
        weight_r2 (float): Weight for R² penalty in the combined score.
        weight_pbias (float): Weight for Percent Bias in the combined score.
        weight_nse (float): Weight for NSE penalty in the combined score.

    Returns:
        float: Combined fitness score (lower is better).
    """
    # Convert inputs to numpy arrays
    obs_rdii = np.array(obs_rdii)
    sim_rdii = np.array(sim_rdii)
    
    # Pad the shorter array with zeros to match the length of the longer one
    max_length = max(len(obs_rdii), len(sim_rdii))
    obs_rdii = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), mode='constant')
    sim_rdii = np.pad(sim_rdii, (0, max_length - len(sim_rdii)), mode='constant')
    
    # Check for invalid outputs
    if sim_rdii.size == 0 or np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
        print("Invalid sim_rdii generated, returning large penalty.")
        return float('inf')

    # Replace NaNs or infinite values with large penalties
    obs_rdii = np.nan_to_num(obs_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
    sim_rdii = np.nan_to_num(sim_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
    
    # Calculate Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((obs_rdii - sim_rdii) ** 2))
    normalized_rmse = rmse / (rmse + 1)  # Normalize RMSE to range [0, 1]
    
    # Calculate R² (Coefficient of Determination)
    ss_total = np.sum((obs_rdii - np.mean(obs_rdii)) ** 2)
    ss_residual = np.sum((obs_rdii - sim_rdii) ** 2)
    r2 = 1 - (ss_residual / ss_total) if ss_total != 0 else 0
    r2_penalty = abs(1 - r2)  # Minimize deviation from 1

    # Calculate Percent Bias (PBIAS)
    pbias = 100 * np.sum(sim_rdii - obs_rdii) / np.sum(obs_rdii) if np.sum(obs_rdii) != 0 else float('inf')
    normalized_pbias = abs(pbias) / (abs(pbias) + 1)  # Normalize PBIAS to range [0, 1]

    # Calculate Nash-Sutcliffe Efficiency (NSE)
    nse = 1 - (ss_residual / ss_total) if ss_total != 0 else -float('inf')
    nse_penalty = abs(1 - max(0, nse))  # NSE below 0 is penalized more

    # Combined fitness score (weighted sum of penalties)
    combined_score = (
        weight_rmse * normalized_rmse +
        weight_r2 * r2_penalty +
        weight_pbias * normalized_pbias +
        weight_nse * nse_penalty
    )

    return combined_score


In [104]:
# def fitness(obs_rdii, sim_rdii, delta_t):
#     """
#     Calculate Percent Bias (PBIAS) and Root Mean Square Error (RMSE) for RDII.

#     Args:
#         obs_rdii (array-like): Observed RDII values.
#         sim_rdii (array-like): Simulated RDII values.
#         delta_t (float): Time interval in seconds.

#     Returns:
#         dict: Dictionary containing PBIAS and RMSE values.
#     """
#     # Convert inputs to numpy arrays
#     obs_rdii = np.array(obs_rdii)
#     sim_rdii = np.array(sim_rdii)
    
#     # Truncate or pad sim_rdii to match the length of obs_rdii
#     if len(sim_rdii) > len(obs_rdii):
#         sim_rdii = sim_rdii[:len(obs_rdii)]
#     elif len(sim_rdii) < len(obs_rdii):
#         sim_rdii = np.pad(sim_rdii, (0, len(obs_rdii) - len(sim_rdii)), mode='constant')
    
#     # Check for invalid outputs
#     if sim_rdii.size == 0 or np.any(np.isnan(sim_rdii)) or np.any(np.isinf(sim_rdii)):
#         print("Invalid sim_rdii generated, returning large penalty.")
#         return float('inf')

#     obs_rdii = np.nan_to_num(obs_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
#     sim_rdii = np.nan_to_num(sim_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
    
#     # Calculate Root Mean Square Error (RMSE)
#     rmse = np.sqrt(np.mean((obs_rdii - sim_rdii) ** 2))

#       # Calculate Percent Bias (PBIAS)
#     #pbias = 100 * np.sum(sim_rdii - obs_rdii) / np.sum(obs_rdii)

#     # Calculate the volumes for observed and simulated RDII (area under the curve)
#     #obs_volume = np.trapz(obs_rdii, dx=delta_t)
#     #sim_volume = np.trapz(sim_rdii, dx=delta_t)
    
#     # Calculate Percent Vol. difference
#     #vol_diff = 100 * (sim_volume - obs_volume) / obs_volume

#     # Calculate Peak Flow Difference
#     #peak_obs = np.max(obs_rdii)
#     #peak_sim = np.max(sim_rdii)
#     #peak_flow_diff = peak_sim - peak_obs
    
#     # Calculate Time to Peak Difference
#     #time_to_peak_obs = np.argmax(obs_rdii) * delta_t  # Time to peak for observed in seconds
#     #time_to_peak_sim = np.argmax(sim_rdii) * delta_t  # Time to peak for simulated in seconds
#     #time_to_peak_diff = (time_to_peak_sim - time_to_peak_obs) / 60  # Convert difference to minutes
    
#     # Return results as a dictionary
#     return rmse
#     #{
#         #"PBIAS": pbias,
#         #"RMSE": rmse,
#         #"Peak Flow Difference (cfs)": peak_flow_diff,
#         #"Time to Peak Difference (min)": time_to_peak_diff,
#         #"Percentage Vol. Difference (cf)": vol_diff
#     #}

# Calculate ojbective function valus for Best parameters

In [107]:
def calculate_criteria(best_params, obs_rdii, delta_t, rainfall, area_acres):
    """
    Calculate and print RMSE, R2, Percent Bias, NSE, and Percent Volume Difference
    for the given best parameters.

    Args:
        best_params (list of tuples): The best parameters as a list of three (R, T, K) tuples.
        obs_rdii (array-like): Observed RDII values.
        delta_t (float): Time interval in seconds.
        rainfall (array-like): Rainfall values.
        area_acres (float): Area in acres.
    """
    # Simulate RDII using the best parameters
    sim_rdii = RDII_calculation(best_params, delta_t, rainfall, area_acres)
    sim_rdii = np.array(sim_rdii)

    # Convert inputs to numpy arrays
    obs_rdii = np.array(obs_rdii)
        
    # Pad the shorter array with zeros to match the length of the longer one
    max_length = max(len(obs_rdii), len(sim_rdii))
    obs_rdii = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), mode='constant')
    sim_rdii = np.pad(sim_rdii, (0, max_length - len(sim_rdii)), mode='constant')

    # # Adjust lengths to match obs_rdii
    # if len(sim_rdii) > len(obs_rdii):  # Truncate if sim_rdii is longer
    #     sim_rdii = sim_rdii[:len(obs_rdii)]
    # elif len(sim_rdii) < len(obs_rdii):  # Pad zeros if sim_rdii is shorter
    #     sim_rdii = np.pad(sim_rdii, (0, len(obs_rdii) - len(sim_rdii)), mode='constant')

    # Replace NaNs or infinite values with large penalties
    obs_rdii = np.nan_to_num(obs_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
    sim_rdii = np.nan_to_num(sim_rdii, nan=0.0, posinf=1e6, neginf=-1e6)

    # Calculate RMSE
    rmse = np.sqrt(np.mean((obs_rdii - sim_rdii) ** 2))

    # Calculate R2
    ss_res = np.sum((obs_rdii - sim_rdii) ** 2)
    ss_tot = np.sum((obs_rdii - np.mean(obs_rdii)) ** 2)
    r2 = 1 - (ss_res / ss_tot if ss_tot != 0 else float('inf'))

    # Calculate Percent Bias (PBIAS)
    pbias = 100 * np.sum(sim_rdii - obs_rdii) / np.sum(obs_rdii) if np.sum(obs_rdii) != 0 else float('inf')

    # Calculate NSE
    nse = 1 - (ss_res / ss_tot if ss_tot != 0 else float('inf'))

    # # Calculate Percent Volume Difference
    # obs_volume = np.trapz(obs_rdii, dx=delta_t)
    # sim_volume = np.trapz(sim_rdii, dx=delta_t)
    # vol_diff = 100 * (sim_volume - obs_volume) / obs_volume if obs_volume != 0 else float('inf')

    # Print the criteria
    print(f"Criteria values for best parameters:")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R2: {r2:.4f}")
    print(f"  Percent Bias (PBIAS): {pbias:.6f}%")
    print(f"  Nash-Sutcliffe Efficiency (NSE): {nse:.4f}")
   # print(f"  Percent Volume Difference: {vol_diff:.2f}%")

    return {
        "RMSE": rmse,
        "R2": r2,
        "PBIAS": pbias,
        "NSE": nse
        #"Vol_Diff": vol_diff
    }

# Calculate criteria Multiple Run for Best parameters

In [110]:
def calculate_criteria_multiple_runs(all_params, obs_rdii, delta_t, rainfall, area_acres, filename=None):
    """
    Calculate and store evaluation criteria (RMSE, R2, Percent Bias, NSE)
    for multiple parameter sets, and compute average values.

    Args:
        all_params (list): List of parameter sets from multiple runs.
        obs_rdii (array-like): Observed RDII values.
        delta_t (float): Time interval in seconds.
        rainfall (array-like): Rainfall values.
        area_acres (float): Area in acres.
        filename (str, optional): Name of the Excel file to save the results. Defaults to None.

    Returns:
        pd.DataFrame: A DataFrame containing evaluation criteria for each parameter set.
    """
    results = []

    for idx, params in enumerate(all_params):
        # Simulate RDII using the current parameter set
        sim_rdii = RDII_calculation(params, delta_t, rainfall, area_acres)
        sim_rdii = np.array(sim_rdii)

        # Convert inputs to numpy arrays
        obs_rdii = np.array(obs_rdii)
        
        # Pad the shorter array with zeros to match the length of the longer one
        max_length = max(len(obs_rdii), len(sim_rdii))
        obs_rdii = np.pad(obs_rdii, (0, max_length - len(obs_rdii)), mode='constant')
        sim_rdii = np.pad(sim_rdii, (0, max_length - len(sim_rdii)), mode='constant')

        # Replace NaNs or infinite values with large penalties
        obs_rdii = np.nan_to_num(obs_rdii, nan=0.0, posinf=1e6, neginf=-1e6)
        sim_rdii = np.nan_to_num(sim_rdii, nan=0.0, posinf=1e6, neginf=-1e6)

        # Calculate RMSE
        rmse = np.sqrt(np.mean((obs_rdii - sim_rdii) ** 2))

        # Calculate R2
        ss_res = np.sum((obs_rdii - sim_rdii) ** 2)
        ss_tot = np.sum((obs_rdii - np.mean(obs_rdii)) ** 2)
        r2 = 1 - (ss_res / ss_tot if ss_tot != 0 else float('inf'))

        # Calculate Percent Bias (PBIAS)
        pbias = 100 * np.sum(sim_rdii - obs_rdii) / np.sum(obs_rdii) if np.sum(obs_rdii) != 0 else float('inf')

        # Calculate NSE
        nse = 1 - (ss_res / ss_tot if ss_tot != 0 else float('inf'))

        # Store the results
        results.append({
            "Run": idx + 1,
            "RMSE": rmse,
            "R2": r2,
            "PBIAS": pbias,
            "NSE": nse
        })

    # Convert results to a DataFrame
    results_df = pd.DataFrame(results)

    # Compute average values
    averages = results_df[["RMSE", "R2", "PBIAS", "NSE"]].mean()

    # Print summary statistics
    print("Summary of criteria across runs:")
    print(results_df.describe())
    print("\nAverage values:")
    print(averages)

    # Add average row to the DataFrame
    averages_row = pd.DataFrame(averages).T
    averages_row["Run"] = "Average"
    results_df = pd.concat([results_df, averages_row], ignore_index=True)

    # Save results to an Excel file if a filename is provided
    if filename:
        results_df.to_excel(filename, index=False)
        print(f"\nResults saved to {filename}")

    return results_df


# Function to plot obs. RDII vs sim. RDII

In [113]:
def plot_rdii(obs_rdii, sim_rdii, delta_t, filename = None):
    """
    Plot observed and simulated RDII on the same plot.

    Args:
        obs_rdii (array-like): Observed RDII values.
        sim_rdii (array-like): Simulated RDII values.
        delta_t (float): Time interval in seconds.
    """
    # Calculate the time values in minutes
    time_values = np.arange(len(obs_rdii)) * delta_t / 3600  # Convert to minutes

    # Plot observed and simulated RDII
    plt.figure(figsize=(12, 6))
    plt.plot(time_values, obs_rdii, label="Observed RDII", color='blue')
    plt.plot(time_values, sim_rdii, label="Simulated RDII", color='orange')
    
    # Plot settings
    plt.xlabel('Time [Hrs]')
    plt.ylabel('RDII [cfs]')
    plt.title('Observed vs. Simulated RDII')
    plt.legend()
    plt.grid(True)
    plt.xlim(left=0)
    plt.ylim(bottom=0)

        # Save the plot if filename is provided
    if filename:
        plt.savefig(filename)
        print(f"Plot saved as {filename}")

        
    # Show plot
    plt.show()